# 音声会話ファイルの文字起こしと、LLM による会話文章の校正

- 文字起こしは、Azure AI Speech Service の speech to text を利用
  - python SDK により呼び出し
  - speech to text 関数は、transcriber_module.py に記載


In [11]:
import os
from transcriber_module import transcribe_audio_file  # 文字起こし関数
from llm_text_corrector import correct_text_with_llm  # 文章修正関数

## 1. 文字起こし


In [ ]:
%%time

audio_file = "./audio_file/audio_long.WAV"  # 音声ファイルのパス
output_file = "./text_file/audi_long_to_text_result.txt"

results = transcribe_audio_file(audio_file, output_file)
# resultsにSpeakerごとの文字起こし結果（リスト）が返る


Starting transcription for: ./audio_file/audio_long.WAV ...
Transcription in progress...
SESSION STARTED: ef4c483fe981496db460b67d08bad804
SESSION STOPPED: ef4c483fe981496db460b67d08bad804
Stopping transcription...
Transcription stopped.
Transcription results saved to: ./text_file/audi_long_to_text_result.txt
CPU times: total: 5.53 s
Wall time: 10min 16s


In [3]:
# 文字起こし結果
with open(output_file, encoding="utf-8") as f:
    ori_text = f.read()

print("-最初の500文字-")
print(ori_text[:500])

-最初の500文字-
Speaker Guest-1: ここから本日はよろしくお願いします。よろしくお願いします。まずは、ええ、原田さんですね。はい、2015年から20年までええ日本銀行政策委員会審議委員を務められた名古屋商科大学ビジネススクール教授の原田さんということですね。よろしくお願いします。そして、小黒さんは元財務官僚で、金融、そして公共経済学などが専門の法政大学教授ということですね。はい、よろしくお願いします。ちょっとお2人の立ち位置と確認したいんですけれど、まずは原田さんは。
Speaker Guest-1: それで黒田路線を継続すべきっていう考えでよろしいですかその通りですはい。そして大倉さんは財政規制、あ、規律派で、これまあ黒田路線からちょっと転換すべき時期じゃないかというお考えということでよろしいですか微修正をしたほうがいいんじゃないか微修正ということでゆっくりですね、ゆっくりそこはお2人、意見として違うということですねピリピリしてますか？なんか、そんな感じですよ。全然してないです。あ、全然知らない。でも、だから経済というのはこ。うやって学者さんクラスといいます
Speaker Guest


### 文字起こし結果確認

- 同じ話し手の文章が 2 つに分けられて別な人の文章に入っているなどの問題あり。
- 話者判別がうまくいっていない箇所がみられる。
- 漢字の誤変換がみられる。

<b>上記問題解消のため、LLM(gpt-o3-mini)に文字起こし文章の校正を依頼する</b>


## 2. 文章校正

- 推論モデル(gpt-o3-mini)を利用
- システムプロンプトは、prompt/system_prompt.txt に記載


In [14]:
%%time

# 各種ファイルパスを指定

# システムプロンプトのファイルパス
# system_prompt_file = "./prompt/system_prompt_1.txt"
# system_prompt_file = "./prompt/system_prompt_2.txt"
# system_prompt_file = "./prompt/system_prompt_3.txt"
system_prompt_file = "./prompt/system_prompt_4.txt"

# 文字起こし結果のファイルパス
user_message_file = "./text_file/audi_long_to_text_result.txt"

# 出力ファイルのパス
# output_file2 = "./modified_text_file/modified_by_o3mini_1.txt"
# output_file2 = "./modified_text_file/modified_by_o3mini_2.txt"
# output_file2 = "./modified_text_file/modified_by_o3mini_3.txt"
output_file2 = "./modified_text_file/modified_by_o3mini_4.txt"


# LLM呼び出し
result = correct_text_with_llm(
    system_prompt_file=system_prompt_file,
    user_message_file=user_message_file,
    output_file=output_file2,
)


-call LLM...-

-fin.-

LLM response written to ./modified_text_file/modified_by_o3mini_4.txt
司会：ここから本日はよろしくお願いします。よろしくお願いします。まずは、ええ、原田さんですね。はい、2015年から20年までええ日本銀行政策委員会審議委員を務められた名古屋商科大学ビジネススクール教授の原田さんということですね。よろしくお願いします。そして、小黒さんは元財務官僚で、金融、そして公共経済学などが専門の法政大学教授ということですね。はい、よろしくお願いします。ちょっとお2人の立ち位置と確認したいんですけれど、まずは原田さんは。

司会：それで黒田路線を継続すべきっていう考えでよろしいですかその通りですはい。そして大倉さんは財政規制、あ、規律派で、これまあ黒田路線からちょっと転換すべき時期じゃないかというお考えということでよろしいですか微修正をしたほうがいいんじゃないか微修正ということでゆっくりですね、ゆっくりそこはお2人、意見として違うということですねピリピリしてますか？なんか、そんな感じですよ。全然してないです。あ、全然知らない。でも、だから経済というのはこ。うやって学者さんクラスといいます

司会：これずっと勉強されてる。2人でもやっぱり意見が違うというのが経済というこ
CPU times: total: 250 ms
Wall time: 1min 9s


#### 結果

- 話者判別の修正が大分ましになっている。
- 一人の会話が二人に分かれている現象も大分解消している。

##### 注意点：

- gpt-4o の場合、余計な要約や勝手な校正が入るため、gpt-o3-mini に変更
- o3-mini では、期待通りの動きをしたが、一部内容の省略がみられるので、プロンプトの修正や gpt4.1 などとも比較必要
- 同じシステムプロンプトを用いても結果が異なる場合が出てくるので、temperature などの調整も必要
- 4 回目の修正版システムプロンプトの結果が 3 回目より悪化したため、細かすぎる指示による不具合が出てきている。丁度いいプロンプト指示文の探索が必要
